In [9]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from LIM import utils as ut
from LIM import LIM_class
import torch
import torch.nn as nn
import torch.optim as optim
import json
import itertools
from torch.utils.data import DataLoader, Dataset

#plt.style.use("../plotting.mplstyle")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:

data = xr.open_dataset("C:/Users/felix/PycharmProjects/ML_Climate_Science_Research_Project/LIM/data/ts_Amon_CESM2_piControl_r1i1p1f1.nc")["ts"]
#data = xr.open_dataset("./data/zos_Amon_CESM2_piControl_r1i1p1f1.nc")["zos"]
#data_old = xr.open_dataset("./data/ssta_1950_2021.nc")["ssta"]
mask = xr.open_dataset("C:/Users/felix/PycharmProjects/ML_Climate_Science_Research_Project/LIM/data/sftlf_fx_CESM2_historical_r1i1p1f1.nc")["sftlf"]

#14400 orginial size
data = data[:200, :, :]

data = ut.apply_mask(mask, data)
#print("Data : {} + shape {}".format(data, data.shape))

data_anomalies = ut.calculate_monthly_anomalies(data)
#print("Month mean : {} + shape : {}".format(data_anomalies, data_anomalies.shape))

data_cropped =ut.crop_xarray(data_anomalies)
#print("Data cropped : {} + shape : {}".format(data_cropped, data_cropped.shape))


pca_10 = ut.SpatioTemporalPCA(data_cropped, n_components=20)
#pca_10 = ut.SpatioTemporalPCA(data_anomalies, n_components=20)
eof_10 = pca_10.eofs()
pc_10 = pca_10.principal_components()

In [11]:
# Create training and test data
data = pca_10.principal_components()
index_train = int(0.8 * len(data["time"]))
data_train = data[:, :index_train]
data_test = data[:, index_train:]

# Creating an example LIM object
tau = 1
model = LIM_class.LIM(tau)
model.fit(data_train.data)
#print("Data train : {} + shape: {} + type: {}".format(data_train.data[:5], data_train.data.shape, type(data_train.data)))

Covariance matrix has negative values!


In [12]:
from FNN_model import *

# Set random seed for reproducibility
torch.manual_seed(42)

def hyperparameter_training_loop(dataloader, input_size, output_size, target_len=4):

    # Generate all possible combinations of hyperparameters
    combinations = list(itertools.product(*hyperparams.values()))

    overall_best_model = {"hidden_size": None,
                         "learning_rate": None,
                         "num_epochs": None,
                         "loss": float('inf')}

    # Iterate over each hyperparameter combination
    for i, params in enumerate(combinations):
        print(f"Testing parameter combination {i+1}/{len(combinations)}: {params}")

        hidden_size = params[0]
        learning_rate = params[1]
        num_epochs = params[2]

        # Train the model
        model = FeedforwardNetwork(input_size, hidden_size, output_size)
        losses = train(model, dataloader, num_epochs, learning_rate, input_size, target_len)

        # Save the model and hyperparameters to a file
        result = {
            'hyperparameters': {
                'hidden_size': params[0],
                'learning_rate': params[1],
                'num_epochs': params[2],
                'best_loss': losses[-1],
                "losses": losses,
            }
        }
        if losses[-1] < overall_best_model["loss"]:
            overall_best_model["hidden_size"] = params[0]
            overall_best_model["learning_rate"] = params[1]
            overall_best_model["num_epochs"] = params[2]
            overall_best_model["loss"] = losses[-1]
            overall_best_model["losses"] = losses

        filename = f"./model_trained/fnn_model_{i+1}.pt"
        with open(filename, 'w') as f:
            json.dump(result, f)

        print(f"Saved model and hyperparameters to {filename}\n")

    return overall_best_model


batch_size = 1
sequence_length = 4
target_len = 4

# Create the DataLoader for first principal component
data = np.array(data)[:, :1000]
print("Data : {} + shape: {} + type: {}".format(data, data.shape, type(data)))
dataloader = create_dataloader(data, batch_size, sequence_length, target_len)

# Hyperparameter search space
hyperparams = {
    'hidden_size': [32, 64],
    'learning_rate': [0.01, 0.001],
    'num_epochs': [3000]
}

#Train the model
input_size =  data.shape[0] * sequence_length
output_size = data.shape[0] * 1

#print("Input size : {}".format(input_size))
#train(model, dataloader, num_epochs, learning_rate)
best_model = hyperparameter_training_loop(dataloader, input_size, output_size)

print(f"Best model: {best_model}")
#plot_loss_evolution(best_model["losses"], np.arange(0,best_model["num_epochs"], dtype=int), best_model["learning_rate"], best_model["hidden_size"])




Data : [[  5.3271937    8.385421    11.919489   ...   0.54701316 -13.858885
  -13.140837  ]
 [ 14.023859    14.754921     9.171422   ...  10.982141    13.614319
   15.255447  ]
 [ -3.981345     5.4285164    4.3772383  ... -14.726523    -5.7204304
   -2.127743  ]
 ...
 [ -6.0170465   -2.3966491   -0.2913321  ...   0.37381372   2.4381244
    2.961527  ]
 [  0.9753295    4.08532     -0.08006045 ...  -4.477956    -1.5456988
   -2.2537868 ]
 [  0.7908709   -1.8077799   -1.2615875  ...   1.1779681   -1.9372445
   -1.7934109 ]] + shape: (20, 200) + type: <class 'numpy.ndarray'>
Testing parameter combination 1/4: (32, 0.01, 3000)


C:\Users\felix\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\autograd\__init__.py:197: UserWarning: Error detected in AddmmBackward0. Traceback of forward call that caused the error:
  File "C:\Users\felix\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\felix\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\felix\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\felix\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 982, in launch_instance
    app.start()
  File "C:\Users\felix\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
    self.io_loop.start()
  File "C:\Users\felix\AppData\Local\Programs\P

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [1, 80]], which is output 0 of AsStridedBackward0, is at version 80; expected version 79 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!